# Segmentation RFM de la Base de données clients

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

In [17]:
# Dl la base de données

df = pd.read_csv('db_inter_avant_RFM.csv')
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
print(df.columns)
print(df.info(verbose = True))

Index(['Unnamed: 0', 'product_id', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty',
       'product_category_name_english', 'order_id', 'customer_id',
       'order_status', 'order_purchase_timestamp', 'customer_unique_id',
       'customer_city', 'customer_state', 'order_item_id', 'seller_id',
       'price', 'freight_value', 'payment_type', 'payment_installments',
       'payment_value', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'year', 'month', 'day'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115223 entries, 0 to 115222
Data columns (total 26 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Unnamed: 0                     115223 non-null  int64         
 1   product_id                     115223 non-null  object        
 2   product_name_lenght            115223 non-null  float64       
 3   product

### Observation des fréquences de chaque clients

In [38]:
df['customer_unique_id'].value_counts()

f9ae226291893fda10af7965268fb7f6    35
6fbc7cdadbb522125f4b27ae9dee4060    34
9a736b248f67d166d2fbb006bcb877c3    30
8af7ac63b2efbcbd88e5b11505e8098a    25
5419a7c9b86a43d8140e2939cd2c2f7e    24
                                    ..
49f3c1a29bc68d07302b330b31c8b28e     1
f15aac4645f84bde58ae87779e6494a5     1
0551e29d87422d88feca4626c030ccef     1
3edbdef5636cc6b1a8c4f661a8910ba2     1
9aa5a8a2a616d12c8b2473429b1bdd1e     1
Name: customer_unique_id, Length: 94087, dtype: int64

### Observation de la plage de temps de la database

In [39]:
print('Orders from {} to {}'.format(df['order_purchase_timestamp'].min(),
                                    df['order_purchase_timestamp'].max()))

Orders from 2016-09-04 21:15:19 to 2018-09-03 09:06:57


In [40]:
df['order_id'].value_counts().head()

fedcd9f7ccdc8cba3a18defedd1a5547    34
fa65dad1b0e818e3ccc5cb0e39231352    25
465c2e1bee4561cb39e0db8c5993aafc    24
c6492b842ac190db807c15aff21a7dd6    24
895ab968e7bb0d5659d16cd74cd1650c    24
Name: order_id, dtype: int64

In [41]:
orders = df.groupby(['order_id', 'order_purchase_timestamp', 'customer_unique_id']).agg({'payment_value': lambda x: x.sum()}).reset_index()
orders.head()

,order_id,order_purchase_timestamp,customer_unique_id,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,2017-09-13 08:59:02,871766c5855e863f6eccc05f988b23cb,72.19
1,00018f77f2f0320c557190d7a144bdd3,2017-04-26 10:53:06,eb28e67c4c0b83846050ddfb8a35d051,259.83
2,000229ec398224ef6ca0657da4fc703e,2018-01-14 14:33:31,3818d81c6709e39d06b2738a8d3a2474,216.87
3,00024acbcdf0a6daa1e931b038114c75,2018-08-08 10:00:35,af861d436cfc08b2c2ddefd0ba074622,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,2017-02-04 13:57:51,64b576fb70d441e8f1b2d7d446e483c5,218.04


In [42]:
# On considère que la date actuelle correspond à l'horaire du dernier achat pour calculer les deltas de temps

NOW = orders['order_purchase_timestamp'].max() + timedelta(days=1)
period = 365 * 2 #Plage des achats = 2 ans
NOW

Timestamp('2018-09-04 09:06:57')

In [43]:
orders['DaysSinceOrder'] = orders['order_purchase_timestamp'].apply(lambda x: (NOW - x).days)

### Calcul de la Frequency et de la Recensy de chaque customer

In [44]:
aggr = {
    'DaysSinceOrder': lambda x: x.min(),  # the number of days since last order (Recency)
    'order_purchase_timestamp': lambda x: len([d for d in x if d >= NOW - timedelta(days=period)]), # the total number of orders in the last period (Frequency)
}
rfm = orders.groupby('customer_unique_id').agg(aggr).reset_index()
rfm.rename(columns={'DaysSinceOrder': 'Recency', 'order_purchase_timestamp': 'Frequency'}, inplace=True)
rfm.head()

,customer_unique_id,Recency,Frequency
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1
2,0000f46a3911fa3c0805444483337064,542,1
3,0000f6ccb0745a6a4b88665a16c9f078,326,1
4,0004aac84e0df4da2b147fca70cf8255,293,1


In [45]:
rfm['Frequency'].value_counts().head()

1    91238
2     2617
3      185
4       30
5        9
Name: Frequency, dtype: int64

### Calcul de la Monetary value de chaque customer

In [47]:
rfm['Monetary'] = rfm['customer_unique_id'].apply(lambda x: orders[(orders['customer_unique_id'] == x) & \
                                                           (orders['order_purchase_timestamp'] >= NOW - timedelta(days=period))]\
                                                           ['payment_value'].sum())
rfm.head()

,customer_unique_id,Recency,Frequency,Monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,27.19
2,0000f46a3911fa3c0805444483337064,542,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,326,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,293,1,196.89


In [49]:
df_i = rfm.to_csv('RFM.csv')

In [3]:
df_RFM = pd.read_csv('RFM.csv')

### Attribution d'une note entre 1 et 5 pour chaque critère RMF par user

In [4]:
quintiles = df_RFM[['Recency', 'Frequency', 'Monetary']].quantile([.2, .4, .6, .8]).to_dict()
quintiles

{'Recency': {0.2: 97.0, 0.4: 182.0, 0.6: 273.0, 0.8: 388.0},
 'Frequency': {0.2: 1.0, 0.4: 1.0, 0.6: 1.0, 0.8: 1.0},
 'Monetary': {0.2: 55.95,
  0.4: 89.75400000000002,
  0.6: 140.02,
  0.8: 238.65800000000004}}

In [6]:
def r_score(x):
    if x <= quintiles['Recency'][.2]:
        return 5
    elif x <= quintiles['Recency'][.4]:
        return 4
    elif x <= quintiles['Recency'][.6]:
        return 3
    elif x <= quintiles['Recency'][.8]:
        return 2
    else:
        return 1

def fm_score(x, c):
    if x <= quintiles[c][.2]:
        return 1
    elif x <= quintiles[c][.4]:
        return 2
    elif x <= quintiles[c][.6]:
        return 3
    elif x <= quintiles[c][.8]:
        return 4
    else:
        return 5    

In [11]:
df_RFM['R'] = df_RFM['Recency'].apply(lambda x: r_score(x))
df_RFM['F'] = df_RFM['Frequency'].apply(lambda x: fm_score(x, 'Frequency'))
df_RFM['M'] = df_RFM['Monetary'].apply(lambda x: fm_score(x, 'Monetary'))

df_RFM.head()


,Unnamed: 0,customer_unique_id,Recency,Frequency,Monetary,R,F,M
0,0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,141.90,4,1,4
1,1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,27.19,4,1,1
2,2,0000f46a3911fa3c0805444483337064,542,1,86.22,1,1,2
3,3,0000f6ccb0745a6a4b88665a16c9f078,326,1,43.62,2,1,1
4,4,0004aac84e0df4da2b147fca70cf8255,293,1,196.89,2,1,4


In [8]:
df_inter = df_RFM.to_csv('RFM_final.csv')